<a href="https://colab.research.google.com/github/RohitPodicheti/Trader-Behavior-Insights/blob/main/Trader_Behavior_Insights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("fear_greed_index.csv")
df.head()

df2=pd.read_csv("historical_data.csv")
df2.head()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df2.describe()

In [ ]:
df2.info()

In [ ]:
df2.isnull().sum()

In [ ]:
df2['trade_datetime'] = pd.to_datetime(df2['Timestamp IST'], format="%d-%m-%Y %H:%M")

In [ ]:
df2.head(3)

In [ ]:
df2['trade_date'] = df2['trade_datetime'].dt.date
df2['Closed PnL'] = pd.to_numeric(df2['Closed PnL'], errors='coerce')
df2.head(3)


In [ ]:
df['date'] = pd.to_datetime(df['date']).dt.date
df.head(3)

In [ ]:


df['date'] = pd.to_datetime(df['date']).dt.date
df['sentiment'] = df['classification'].apply(lambda x: 'Fear' if 'Fear' in x else 'Greed')


In [ ]:
trader_clean = df2[['trade_date', 'Side', 'Closed PnL', 'Size USD', 'Execution Price', 'Start Position']].dropna()
sentiment_clean = df[['date', 'sentiment']]

In [ ]:
merged_df = pd.merge(trader_clean, sentiment_clean, left_on="trade_date", right_on='date', how='inner')


In [ ]:
merged_df.head()

In [ ]:
print(df2.columns)


In [ ]:
merged_df['profitable'] = (merged_df['Closed PnL'] > 0).astype(int)

In [ ]:

print(merged_df.describe())


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="whitegrid")

plt.figure(figsize=(6,4))
sns.countplot(x='profitable', data=merged_df)
plt.title("Trade Profitability Distribution")
plt.xticks([0,1], ['Not Profitable', 'Profitable'])
plt.xlabel("Profitability")
plt.ylabel("Number of Trades")
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(6,4))
sns.barplot(x='sentiment', y='profitable', data=merged_df)
plt.title("Average Profitability by Market Sentiment")
plt.xlabel("Sentiment")
plt.ylabel(" Profitable Trades")
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(10,4))
sns.histplot(data=merged_df, x='Size USD', hue='sentiment', element='step', bins=30)
plt.title("Trade Size Distribution by Sentiment")
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le_side = LabelEncoder()
le_sentiment = LabelEncoder()

In [ ]:
merged_df['side_encoded'] = le_side.fit_transform(merged_df['Side'])
merged_df['sentiment_encoded'] = le_sentiment.fit_transform(merged_df['sentiment'])


In [ ]:
# Features and target
X = merged_df[['side_encoded', 'sentiment_encoded', 'Size USD', 'Execution Price', 'Start Position']]
y = merged_df['profitable']


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)


In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\n Classification Report:\n", classification_report(y_test, y_pred))
print("\n Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
